# Batch Processing

Here, we will use a jupyter notebook in a non-traditional way just to keep the workshop consistent. We are going to run our code in a single cell. 

In [ ]:
'''
0. User Input

This script now only processes data for one of the two hypotheses at a time. Remember, these hypotheses are as follows:
Hypothesis 1: T1 vs T2
Hypothesis 2: T1 vs T3
The reason for this is that some participants have data for one of the hypotheses, but not the other and adapting this code to account 
for these differences would make it more complex than I want it to be. As such, designate which hypothesis you want to process below.
'''

hypothesis = 2 #Either 1 or 2

'''
1. Setting Up Environment

We will begin by setting up our environment and finding the files to process.
'''

'''
1.1. Import Modules

First, we must import the modules we will use in this tutorial.
'''

import warnings
warnings.filterwarnings('ignore')
import os #For directory information extraction
import numpy as np #For basic computations
import mne #Main EEG analysis package
import pymatreader #MNE depencency for EEGLab files
import scipy.fft #To conduct FFT
from ipywidgets import IntProgress
from IPython.display import display

'''
1.2. Defining Extraction Function

The following function is going to be the code that extracts the EEG features to be used as predictors in your model. 
I wanted to pull this function out from the main loop below to be explicit that this function is where you do your main modifications. 
In this function, I conduct FFT analyses as an example of what you can do, but the intent is for you to replace this with the analyses you are interested in.

'''

def featureExtraction(EEG):
    '''
    Input:
        EEG: An MNE EEG structure. This assumes pre-processed data, but might work with raw data (untested)
        
    Output:
        timeFeatures: A numpy array of size n, depending on the number of features you want to extract
    '''

    '''
    1.2.1. Determining Our Frequency Resolution
    
    The output of our FFT will ultimately be linked to different frequencies. Although it may be a bit abstract right now, 
    we will compute our frequency resolution ahead of running the FFT as it will make a few of the FFT transformation steps below easier. 
    We will determine the number of datapoints being transformed into FFTs, determine our frequency resolution, 
    and then create an array of frequencies, which will correspond to our FFT output.
    '''

    numberDataPoints = EEG.get_data().shape[2] #Determine how many datapoints will be transformed per trial and channel
    frequencyResolution = EEG.info['sfreq']/numberDataPoints #Determine frequency resolution
    fftFrequencies = np.arange(frequencyResolution,(EEG.info['sfreq']/2),frequencyResolution) #Determine array of frequencies

    '''
    1.2.2. FFT Processing Across Trials and Electrodes
    
    Here, we will loop through our trials and channels for the current file, computing a FFT each time.
    '''

    FFT = np.zeros((EEG.get_data().shape[0],EEG.get_data().shape[1],int(EEG.get_data().shape[2]/2)-1)) #Create empty array
    for trialIndex in range(EEG.get_data().shape[0]): #Cycle through
        for channelIndex in range(EEG.get_data().shape[1]): #Cycle through channels
            fftOutput = None #Empty variable
            fftOutput = scipy.fft.fft(EEG.get_data()[trialIndex,channelIndex,:]) #Compute the Fourier transform
            fftOutput = fftOutput/numberDataPoints #Normalize output
            fftOutput = np.abs(fftOutput) #Absolute transformation
            fftOutput = fftOutput[range(int(numberDataPoints/2))] #Extract the one-sided spectrum
            fftOutput = fftOutput*2 #Double values to account for lost values         
            fftOutput = fftOutput**2 #Convert to power
            fftOutput = fftOutput[1::] #Remove DC Offset
            FFT[trialIndex,channelIndex,:] = fftOutput #Assign fft to dataframe after converting for power and multiplying by two to account for the lost negative aspect

    '''
    1.2.3. Extracting Depression Biomarkers
    
    Now that we have computed all of the FFTs, we can use our results and extract EEG biomarkers of depression. 
    In this workshop, we will focus on the Alpha frequency band. We will extract four biomarkers: Frontal Alpha Amplitude, 
    Parietal Alpha Amplitude, Frontal Alpha Assymetry, and Parietal Alpha Assymetry.
    Let's first define the criteria to be used to extract our biomarkers.
    '''
    
    #Alpha band criteria
    alphaBand = [8,13.5]
    alphaIndex = [np.where(alphaBand[0]==fftFrequencies)[0][0], np.where(alphaBand[1]==fftFrequencies)[0][0]]
        
    #Electrode criteria
    electrodes = np.array(['F7','F8','P7','P8'])
    electrodeIndex = []
    for currentElectrode in range(len(electrodes)):
        electrodeIndex.append(np.where(electrodes[currentElectrode]==np.array(EEG.ch_names))[0][0])
        
    '''
    1.2.4. Extract Four Alpha Quadrants
    
    We will here extract alpha activity from four quadrants of the scalp to be used in the creation of our biomarkers. 
    Specifically, we will extract alpha activity in the frontal left, frontal right, parietal left, and parietal right locations of the head. 
    This code will average the selected electrodes and alpha band, leaving us with alpha power for each trial in each quadrant.
    '''

    frontalLeft = np.mean(FFT[:,electrodeIndex[0],alphaIndex[0]:alphaIndex[1]+1],axis = (0,1))
    frontalRight = np.mean(FFT[:,electrodeIndex[1],alphaIndex[0]:alphaIndex[1]+1],axis = (0,1))
    parietalLeft = np.mean(FFT[:,electrodeIndex[2],alphaIndex[0]:alphaIndex[1]+1],axis = (0,1))
    parietalRight = np.mean(FFT[:,electrodeIndex[3],alphaIndex[0]:alphaIndex[1]+1],axis = (0,1))
          
    '''
    1.2.5. Compute Biomarkers
    
    We will now use the four quadrants that we just extracted in different computations to compute our biomarkers. 
    Again, our biomarkers are frontal alpha amplitude, parietal alpha amplitude, frontal alpha assymetry, 
    and parietal alpha assumetry. For the amplitude biomarkers, we will average the corresponding quadrants 
    (e.g., frontal alpha amplitude will require averaging frontal left and frontal right quadrants), 
    and for the assymetry biomarkers, we will find the difference between the corresponding quadrants 
    (e.g., frontal alpha assymetry will require subtracting the frontal right from the frontal left quadrants). 
    '''

    frontalAlphaAssymetry = (frontalRight-frontalLeft)/(frontalRight+frontalLeft)
    parietalAlphaAssymetry = (parietalRight-parietalLeft)/(parietalRight+parietalLeft)

    '''
    1.2.6. Concatenate Biomarkers

    We have now extracted the EEG features to be used in the next portion of our tutorial, 
    but still need to package it nicely for use. We will stack the EEG features into a matrix alongside a recording ID 
    that signifies which recording session the data is from (i.e., T1 - T4). 
    '''

    timeFeatures = np.stack((frontalAlphaAssymetry,parietalAlphaAssymetry))
    
    return timeFeatures

'''
1.3. Finding Data Files

There are many ways to determine a list of filenames that will be loaded. 
Here, we opted to simply extract any filename that begins with the tag 0372. 
This assumes that you are currently in the directory that contains the data.
'''

#Load outcome data
outcomeFile = '/gpfs/data/brainstorm-ws/data/VALIDATION/VALIDATION_Demographics and Clinical Outcomes_All Series.csv'
outcomeData = np.asarray(np.genfromtxt(outcomeFile, delimiter=',', skip_header = 1, encoding='utf-8-sig'))
outcomeColumns = ('TMSID', 'Series', 'Sex', 'AgeTMSstart', 'SevHxDep', 'outcome')

#Extract participant IDs that will be cycled through
participantIDs = outcomeData[:,0].astype(int).astype(str)
participantIDs = np.char.zfill(participantIDs, 4)
participantSeries = outcomeData[:,1].astype(int).astype(str)

#Determine where the data lives and the three subfolders (note here we are not considering T4)
dataPath = '/gpfs/data/brainstorm-ws/data/VALIDATION/series_'

if hypothesis == 1:
    folders = ['T1','T2']
else:
    folders = ['T1','T3']   

filenames = [] #Create an empty variable
for participantIdx in range(len(participantIDs)): #Cycle through all participants
    participantFilenames = []
    for currentFolder in folders: #Cycle through the four folders
        currentPath = dataPath+participantSeries[participantIdx]+'/'+currentFolder+'/preprocessed/' #Create a path for the specific folder
        allFiles = os.listdir(currentPath) #Retrieve a list of all files within the folder
        currentFilename = list(filter(lambda f: f.startswith(participantIDs[participantIdx]), allFiles)) #Extract the files for the specific participant 
        if currentFilename:
            participantFilenames.append(currentPath+currentFilename[0]) #Add the filename with path to a list to be used when loading data
    
    if len(participantFilenames) == 2:
        for i in range(2):
            filenames.append(participantFilenames[i]) #Add the filename with path to a list to be used when loading data
            
'''
2. The Main loop

Here we loop through all of the files to fully process our data, pulling from our function defined from above in step 1.2.
'''

#Create empty array to append features
EEGFeatures = []
fileIndex = 0
f = IntProgress(min=0, max=(71)) # instantiate the bar
display(f) # display the bar
for participantIndex in range(int(len(filenames)/2)): 
    #Progress report
    #print('Complete: '+ str(round((((participantIndex+1)/int(len(filenames)/2))*100),2))+ '%')
    f.value += 1
    
    participantFeatures = []
    for timeIndex in range(2):
        #Load Data
        EEG = mne.io.read_epochs_eeglab(filenames[fileIndex], verbose = 0)

        #Call extraction function
        timeFeatures = featureExtraction(EEG)
        
        #Concatenate data across time points
        participantFeatures.append(timeFeatures)
        
        #Increase index count
        fileIndex += 1        
    
    TXvsT1 = (np.array(participantFeatures[1]) - np.array(participantFeatures[0]))/(np.array(participantFeatures[1]) + np.array(participantFeatures[0]))

    currentFeatures = TXvsT1 #Combine outcome and features
    EEGFeatures.append(currentFeatures.tolist()) #Add it to the features matrix
    
'''
#3. Save Features as CSV

As a final step, we will save our features as a CSV file for future use. 
'''

if hypothesis == 1:
    np.savetxt('williamsBrainstormChallenge_T1vsT2ValidationOutcomes.csv', EEGFeatures, delimiter=",", comments = '')
else:
    np.savetxt('williamsBrainstormChallenge_T1vsT3ValidationOutcomes.csv', EEGFeatures, delimiter=",", comments = '')

In [ ]:
'''
#4. Predict Outcomes of Validation Set

In addition to the above, we can use our determined equation to predict outcomes of the validation set
'''

#Determine hypothesis (see above for description)
hypothesis = 2

#Import Modules
import numpy as np

#Create Operation Functions
def sigmoid(x):
    return(1/(1 + np.exp(-x)))
    
#Conduct Computations
if hypothesis == 1: 
    
    print('No equation set, please add equation to continue')
    
else:
    #Load Data
    EEGFeatures = np.genfromtxt('williamsBrainstormChallenge_T1vsT3ValidationOutcomes.csv', delimiter=',')
    
    #Create Matrix to be Populated
    predictedOutcomes = np.zeros((len(EEGFeatures),2))
    
    #Cycle Through Participants
    for participant in range(len(EEGFeatures)):
        #EEG Features
        x1 = EEGFeatures[participant][0]
        x2 = EEGFeatures[participant][1]
        
        #Equation
        k1 = np.sin((2.00*x2) + -2.30)
        y1 = (-2.14*k1) + 0.11291785
        
        #Transform
        outcome = sigmoid(y1)
        
        #Allocate to Matrix
        predictedOutcomes[participant,0] = participantIDs[participant]
        predictedOutcomes[participant,1] = round(outcome)
        
    #Save to CSV
    np.savetxt('williamsBrainstormChallenge__T1T3(H2)PredictedOutcomes.csv', predictedOutcomes, delimiter=",", comments = '')
    
#Print Results
print(predictedOutcomes)